In [3]:
import sys
sys.path.append('../build')
import numpy as np
import libry as ry
import time


from lib.rai.rai_env import RaiEnv
from lib.prediction.color_seg import find_ball


In [4]:
#########################################
# initialize rai simulation here        #
#########################################
Rai = RaiEnv(tau=.01,
             realEnv="scenarios/project_env.g",
             modelEnv="scenarios/project_env.g",
             useROS=False,
             initSim=False,
             initConfig=True,
             simulatorEngine=ry.SimulatorEngine.bullet,
             verboseSim=1)

#########################################
# init simulation                       #
#########################################
Rai._init_simulation()
